# Face detection and recognition inference pipeline

In [4]:
! pwd

/home/rohan/cv/repo/facenet_pytorch/examples


In [6]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os
import cv2
from utils.detect_face import extract_face

workers = 0 if os.name == 'nt' else 4

#### Determine if an nvidia GPU is available

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


#### Define MTCNN module
Face Detection model from images.

In [8]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### Define Inception Resnet V1 module

Inception Resnet to get embeddings of each face.

In [9]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

#### Define a dataset and data loader


In [11]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('../../data/sr_img/')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

#### Perfom MTCNN facial detection


In [12]:
aligned = []
names = []
for x, y in loader:
    boxes, prob = mtcnn.detect(x)
    
    for idx, box in enumerate(boxes):
        face = extract_face(x, box, 160, 0, f"../../data/faces/face{idx}.jpg")

After manual segregation into folders

In [13]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('../../data/face_folder')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

In [14]:
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

Face detected with probability: 0.999467
Face detected with probability: 0.999893
Face detected with probability: 0.999949
Face detected with probability: 0.999420
Face detected with probability: 1.000000
Face detected with probability: 0.999997
Face detected with probability: 0.999999
Face detected with probability: 0.995827
Face detected with probability: 0.999962
Face detected with probability: 0.999832
Face detected with probability: 0.999917
Face detected with probability: 0.999829


Save the faces to a folder.

In [15]:
for idx, a in enumerate(aligned[:]):
#     plt.imshow(a.cpu().numpy().transpose(1, 2, 0))
    final_face = a.cpu().numpy().transpose(1, 2, 0)[:,:, [2, 1, 0]]
    cv2.imwrite(f"../../data/faces/face{idx}.jpg", (final_face)*255)

#### Calculate image embeddings

In [16]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

#### Print distance matrix for classes

In [17]:
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
df = pd.DataFrame(dists, columns=names, index=names)
print(df[df["mbappe_real_image"] > 0]["mbappe_real_image"].argmin())

face1


/home/rohan/cv/super-resolution/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
print(df)

                      face1    face10    face11     face2     face3     face4  \
face1              0.000000  1.541790  1.254309  1.311747  1.153131  1.076305   
face10             1.541790  0.000000  1.469768  1.272414  1.415111  1.450225   
face11             1.254309  1.469768  0.000000  1.382727  1.420647  1.491348   
face2              1.311747  1.272414  1.382727  0.000000  1.300457  1.174065   
face3              1.153131  1.415111  1.420647  1.300457  0.000000  1.029869   
face4              1.076305  1.450225  1.491348  1.174065  1.029869  0.000000   
face5              1.437752  1.285775  1.433558  1.226325  1.428909  1.165871   
face6              1.360260  1.427959  1.313521  1.275958  1.499547  1.376816   
face7              1.450648  1.481493  1.407792  1.468432  1.221348  1.331831   
face8              1.362209  1.191827  0.899785  1.442619  1.439378  1.508914   
face9              1.201638  1.487277  1.440368  1.240009  1.236576  1.243151   
mbappe_real_image  0.516229 

In [66]:
from PIL import Image
import matplotlib.pyplot as plt
import cv2